## Neural Network DDI

In [1]:
import numpy as np
from os import listdir
import pickle
from xml.dom.minidom import parse
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from keras.models import Model, Input
from keras.layers import *
#from keras_contrib.layers import CRF

stopwords = set(stopwords.words("english"))
from evaluator import *

%matplotlib inline

## Functions Learner

`load_data()`- Use XML parsing and tokenization functions from previous exercises. Adding a PoS tagger or lemmatizer may be useful. \
Masking the target drugs as e.g. `<DRUG1>`, `<DRUG2>`, and the rest as `<DRUG_OTHER>` will help the algorithm generalize and avoid it focusing in the drug names, which are not relevant for the DDI task (and also make it easier for it to spot the target entities).

In [10]:
def load_data(datadir):
    '''
    Task: Load XML files in given directory, tokenize each sentence, and extract
    learning examples (tokenized sentence + entity pair).
    
    Input: 
        datadir: A directory containing XML files
        
    Output: A list of classification cases. Each case is a list containing sentenceid, entity1_id, entity2_id,
            ground truth relation label, and a list of sentence tokens (each token containing any needed information:
            word, lemma, PoS, offsets, etc.
    '''
    dataset = []
    wordnet_lemmatizer = WordNetLemmatizer()

    for f in listdir(datadir):
        
        tree = parse(datadir + "/" + f)
        sentences = tree.getElementsByTagName("sentence")

        for s in sentences: 
            sid = s.attributes["id"].value   
            stext = s.attributes["text"].value
            stext = stext.replace("-"," ")
            
            ents = {}
            entities = s.getElementsByTagName("entity")
            for e in entities:
                e_id = e.attributes["id"].value
                offset = e.attributes["charOffset"].value
                name = e.attributes["text"].value
                ents[e_id] = name

            pairs = s.getElementsByTagName("pair")
            for p in pairs:
                pair_sent = stext
                e1_id = p.attributes['e1'].value
                e2_id = p.attributes['e2'].value
                if p.attributes['ddi'].value == 'true':
                    ddi = p.attributes['type'].value
                else:
                    ddi = 'null'
                
                aux = [sid, e1_id, e2_id, ddi]
                for key, item in ents.items():
                    if key == e1_id:
                        drug_1 = item
                        pair_sent = pair_sent.replace(drug_1, 'DRUG1')
                    elif key == e2_id:
                        drug_2 = item
                        pair_sent = pair_sent.replace(drug_2, 'DRUG2')
                    else:
                        other_drug = item
                        pair_sent = pair_sent.replace(other_drug, 'DRUGOTHER')
        
                punct = [".",",",";",":","?","!", "'"] # TODO: Change all punctuations to <PUNCT> and stopwords to <SW>
                pair_sent = pair_sent.replace("(", "") # Remove Parenthesis
                pair_sent = pair_sent.replace(")", "") # Remove Parenthesis
                tokens = word_tokenize(pair_sent)
                aux2 = []
                for t in tokens:
                    if t in punct:
                        tk = ('<PUNCT>', '<PUNCT>')
                    elif t in stopwords:
                        tk = ('<SW>', '<SW>')
                    else:
                        t_l = t.lower()
                        lemma = wordnet_lemmatizer.lemmatize(t_l)
                        tk = (t, lemma) # TODO: Add PoS Tag and maybe use stemmer instead of lemmatizer
                    aux2.append(tk)
                aux.append(aux2)
                dataset.append(aux)
    
    return dataset

In [15]:
train_path = '../../labAHLT/data/train'
devel_path = '../../labAHLT/data/devel'
dataset = load_data(train_path)
val_dataset = load_data(devel_path)
print(dataset[65])

['DDI-DrugBank.d234.s2', 'DDI-DrugBank.d234.s2.e0', 'DDI-DrugBank.d234.s2.e1', 'effect', [('Particular', 'particular'), ('caution', 'caution'), ('<SW>', '<SW>'), ('necessary', 'necessary'), ('<SW>', '<SW>'), ('using', 'using'), ('DRUG1', 'drug1'), ('<SW>', '<SW>'), ('cases', 'case'), ('<SW>', '<SW>'), ('mixed', 'mixed'), ('drug', 'drug'), ('overdosage', 'overdosage'), ('since', 'since'), ('<SW>', '<SW>'), ('toxic', 'toxic'), ('effects', 'effect'), ('<SW>', '<SW>'), ('<SW>', '<SW>'), ('convulsions', 'convulsion'), ('<SW>', '<SW>'), ('cardiac', 'cardiac'), ('dysrhythmias', 'dysrhythmias'), ('<SW>', '<SW>'), ('<SW>', '<SW>'), ('drugs', 'drug'), ('taken', 'taken'), ('<SW>', '<SW>'), ('overdose', 'overdose'), ('especially', 'especially'), ('DRUG2', 'drug2'), ('may', 'may'), ('emerge', 'emerge'), ('<SW>', '<SW>'), ('<SW>', '<SW>'), ('reversal', 'reversal'), ('<SW>', '<SW>'), ('<SW>', '<SW>'), ('DRUGOTHER', 'drugother'), ('effect', 'effect'), ('<SW>', '<SW>'), ('DRUGOTHER', 'drugother'), ('<P

In [11]:
def create_index(dataset, max_length):
    '''
    Task: Create index dictionaries both for input (words) and output (labels) from given dataset
    Input: 
        dataset: dataset produced by load_data.
        max_length: maximum length of a sentence (longer sentences will be cut, shorter ones will be padded).
        
    Output: A dictionary where each key is an index name (e.g. "words", "labels"), and the value is a 
            dictionary mapping each word/label to a number. An entry with the value for maxlen is also stored
    Example: 
        >>> create_indx(traindata)
        {'words': {'<PAD>':0, '<UNK>':1, '11-day':2, 'murine':3, 'criteria':4,
                   'stroke':5, ... ,'levodopa':8511, 'terfenadine': 8512}
         'labels': {' null ':0, 'mechanism ':1, 'advise ':2, 'effect ':3, 'int ':4}
         'maxlen': 100 }
    '''
    index_words = {'<PAD>':0, '<UNK>':1}
    i = 2
    
    index_labels = {}
    j = 0

    # We can create other indexes (lemmas, PoS, etc)  
    
    for s in dataset:
        words = s[4]
        label = s[3]
        if label not in index_labels:
            index_labels[label] = j
            j += 1
        for tup in words:
            w = tup[0]
            if w not in index_words:
                index_words[w] = i
                i += 1
    idx = {'words': index_words, 'labels':index_labels, 'maxlen':max_length}     
    
    return idx
    
# Add '<PAD>': 0 and '<UNK>':1 codes to 'words' index. The coding of the rest of the words/labels is arbitrary.
# You may add to the dictionary entries with indexes for other elements you want to use (lemmas, PoS, etc).

In [16]:
idx = create_index(dataset, 100)
#print(idx["words"])
#print(idx["labels"])
#print(idx['maxlen'])

In [12]:
def encode_words(dataset, idx):
    '''
    Task: Encode the words in a sentence dataset formed by lists of tokens into lists of indexes
          suitable for NN input.
    Input: 
        dataset: A dataset produced by load_data.
        idx: A dictionary produced by create_indexs, containing word and label indexes, as well
             as the maximum sentence length.
             
    Output: The dataset encoded as a list of sentence, each of them is a list of word indices.
            If the word is not in the index, <UNK> code is used. If the sentence is shorter than
            max_len it is padded with <PAD> code.
    Example: 
        >>> encode_words(traindata, idx)
            [ [6882 1049 4911 ... 0 0 0 ]
            [  2290 7548 8069 ... 0 0 0 ]
               ...
            [  2002 6582 7518 ... 0 0 0 ] ]
    '''
    max_length = idx['maxlen']
    seq = []
    for s in dataset:
        words = s[4]
        aux = []
        for tup in words:
            w = tup[0]
            if w in idx['words']:
                i = idx['words'][w]
            else:
                i = idx['words']['<UNK>']
            aux.append(i)
        seq.append(aux)
        
    seq_padded = pad_sequences(maxlen = max_length, sequences = seq, padding = 'post')
    
    return seq_padded
        
        
def encode_labels(dataset, idx):
    '''
    Task: Encode the ground truth labels in a dataset of classification examples (sentence + entity pair).
    Input:
        dataset: A dataset produced by load_data.
        idx: A dictionary produced by create_index, containing word and label indexes, as well as the maximum length.
        
    Output: The dataset encoded as a list DDI labels, one per classification example. 
    
    Example :
     >>> encode_labels ( traindata , idx )
     [  [0] [0] [2] ... [4] [0] [0] [1] [0] ]
     [  [4] [6] [4] [4] [4] [4] ... [0] [0] ]
     [  [4] [4] [8] [4] [6] [4] ... [0] [0] ]
         ...
     [  [4] [8] [9] [4] [4] [4] ... [0] [0] ]
     ] 
    '''
    max_length = idx['maxlen']
    seq = []
    seq = [idx['labels'][s[3]] for s in dataset] # Changed [[idx['labels'][s[3]]] for s in dataset]
    Y = [to_categorical(i, num_classes=5) for i in seq]
    Y = np.array(Y)
    
    return Y

In [17]:
X_train = encode_words(dataset, idx)
X_val = encode_words(val_dataset, idx)
print(X_train)

[[   2    3    4 ...    0    0    0]
 [  15   16   17 ...    0    0    0]
 [  15   16   17 ...    0    0    0]
 ...
 [1198  996    6 ...    0    0    0]
 [1198  996    6 ...    0    0    0]
 [1198  996    6 ...    0    0    0]]


In [18]:
Y_train = encode_labels(dataset, idx)
Y_val = encode_labels(val_dataset, idx)
print(Y_train.shape)
print(Y_val.shape)

(23148, 5)
(4669, 5)


In [20]:
## GloVe
# http://nlp.stanford.edu/data/glove.6B.zip
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index['words']) + 1  
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index['words']:
                idx = word_index['words'][word] 
                embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [21]:
glove_path = "../../labAHLT/data/glove.6B/glove.6B.50d.txt"
embedding_matrix = create_embedding_matrix(glove_path, idx, 16)

In [47]:
def build_network(idx, embedding_dim, embedding_matrix):
    '''
    Task: Create network for the learner. 
    Input:
        idx: index dictionary with word/labels codes, plus maximum sentence length.
    Output: Returns a compiled Keras neural network with the specified layers
    '''
    
    #sizes
    n_words = len(idx['words'])
    n_labels = len(idx['labels'])
    max_len = idx['maxlen']
    
    # create network layers
    inp = Input(shape=(max_len,))
    emb_layer = Embedding(input_dim=n_words+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len)(inp) # TransferLearning -> GloVe, BERT
    model = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.4))(emb_layer)
    #model = Conv1D(64, 5, activation='relu')(model)
    model = GlobalMaxPooling1D()(model)
    model = Dense(24, activation='relu')(model)
    out = Dense(n_labels, activation='softmax')(model)
    
    # create and compile model
    model = Model(inp, out)
    
    optimiz = Adam(lr=0.01, decay=1e-6)
    model.compile(optimizer=optimiz, loss="categorical_crossentropy", metrics=["accuracy"]) 

    return model

In [45]:
def build_network(idx, embedding_dim, embedding_matrix):

    #sizes
    n_words = len(idx['words'])
    n_labels = len(idx['labels'])
    max_len = idx['maxlen']
    
    # create network layers
    inp = Input(shape=(max_len,))
    emb_layer = Embedding(input_dim=n_words+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len)(inp) # TransferLearning -> GloVe, BERT
    model = Conv1D(128, 5, activation='relu')(emb_layer)
    model = GlobalMaxPooling1D()(model)
    model = Dense(24, activation='relu')(model)
    out = Dense(n_labels, activation='softmax')(model)
    
    # create and compile model
    model = Model(inp, out)
    
    optimiz = Adam(lr=0.01, decay=1e-6)
    model.compile(optimizer=optimiz, loss="categorical_crossentropy", metrics=["accuracy"]) 

    return model

In [40]:
model = build_network(idx, 16, embedding_matrix)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 16)           79760     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 128)           10368     
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 32)            18560     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 24)                792       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 125 

In [57]:
history = model.fit(X_train, Y_train,
                   batch_size=16, #16/64
                   epochs=2,
                   verbose=1, 
                   validation_data=(X_val, Y_val))

In [24]:
def save_model_and_indexes(model, idx):
    '''
    Task: Save given model and indexs to disk
    Input: 
        model: Keras model created by _build_network, and trained.
        idx: A dictionary produced by create_indexs, containing word and label indexes, 
             as well as the maximum sentence length. 
        filename: Saves the mode into filename.nn and the indexes into filename.idx
    '''
    model.save("ddi-nn.nn")
    
    file = open("index.pkl", "wb")
    pickle.dump(idx, file)
    file.close()

In [23]:
save_model_and_indexes(model, idx)

INFO:tensorflow:Assets written to: ddi-nn.nn/assets


## `Learner()`

In [46]:
def learner(traindir, validationdir):
    '''
    Learns a NN model using traindir as training data, and validationdir as validation data.
    Saves learnt model in a file named modelname
    '''
    # load train and validation data in a suitable form
    traindata = load_data(traindir)
    valdata = load_data(validationdir)
    
    # create indexes from trainindg data 
    max_len = 100
    idx = create_index(traindata, max_len)
    embedding_matrix = create_embedding_matrix(glove_path, idx, 16)
    
    # build network 
    model = build_network(idx, 16, embedding_matrix)
    
    # encode datasets
    Xtrain = encode_words(traindata, idx)
    Ytrain = encode_labels(traindata, idx)
    Xval = encode_words(valdata, idx)
    Yval = encode_labels(valdata, idx)
    
    # train model
    model.fit(Xtrain, Ytrain,
              batch_size=26,
              verbose=1,
              epochs=6,
              validation_data=(Xval, Yval))
    
    #return history
    
    #save model and indexs, for later use in prediction
    save_model_and_indexes(model, idx)

In [47]:
train_path = '../../labAHLT/data/train'
devel_path = '../../labAHLT/data/devel'

learner(train_path, devel_path)

Epoch 1/6
891/891 [==============================] - 4s 4ms/step - loss: 0.4973 - accuracy: 0.8542 - val_loss: 0.5092 - val_accuracy: 0.8068
Epoch 2/6
891/891 [==============================] - 3s 4ms/step - loss: 0.2686 - accuracy: 0.9007 - val_loss: 0.4451 - val_accuracy: 0.8447
Epoch 3/6
891/891 [==============================] - 3s 4ms/step - loss: 0.1956 - accuracy: 0.9288 - val_loss: 0.5073 - val_accuracy: 0.8124
Epoch 4/6
891/891 [==============================] - 4s 4ms/step - loss: 0.1752 - accuracy: 0.9375 - val_loss: 0.4546 - val_accuracy: 0.8563
Epoch 5/6
891/891 [==============================] - 4s 4ms/step - loss: 0.1576 - accuracy: 0.9475 - val_loss: 0.4796 - val_accuracy: 0.8353
Epoch 6/6
891/891 [==============================] - 4s 4ms/step - loss: 0.1393 - accuracy: 0.9530 - val_loss: 0.5414 - val_accuracy: 0.8188
INFO:tensorflow:Assets written to: ddi-nn.nn/assets


In [28]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')
# Need more epochs but it seems to work

***
## Classifier Functions

In [5]:
def load_model_and_indexs():
    '''
    Task: Load model and associate indexs from disk.
    Input:
        filename: filename to be loaded
    Output: Loads a model from filename.nn and its indexes from filename.idx
            Returns the loaded model and indexes. 
    '''
    
    model = load_model("ddi-nn.nn")
    index = open("index.pkl", "rb")
    idx = pickle.load(index)
    
    return model, idx

In [6]:
def output_interactions(dataset, preds, outf): 
    '''
    Task: Output detected DDIs in the format expected by the evaluator.
    Input:
        dataset: A dataset produced by load_data
        preds: Fore each sentence in dataset, a label for its DDI type (or 'null' if no DDI detected)
        
    Output: prints the detected interactions to dtdout in the format required by the evaluator.
    Example:
        >>> output_interactions ( dataset , preds )
            DDI-DrugBank.d398.s0|DDI-DrugBank.d398.s0.e0|DDI-DrugBank.d398.s0.e1|effect
            DDI-DrugBank.d398.s0|DDI-DrugBank.d398.s0.e0|DDI-DrugBank.d398.s0.e2|effect
            DDI-DrugBank.d211.s2|DDI-DrugBank.d211.s2.e0|DDI-DrugBank.d211.s2.e5|mechanism
            ...
    '''
    length = len(dataset)
    for i in range(length):
        sid = dataset[i][0]
        id_e1 = dataset[i][1]
        id_e2 = dataset[i][2]
        ddi_type = preds[i]
        outf.write(str(sid) +"|"+ str(id_e1) +"|"+ str(id_e2) +"|"+ str(ddi_type))
        outf.write("\n")

## `Classifier()`

In [7]:
def predict(datadir, outfile):
    '''
    Loads a NN model from a file 'modelname' and uses it to extract drugs in datadir. Saves
    results to 'outfile' in the appropriate format
    '''
    
    # load model and associated encoding data
    model, idx = load_model_and_indexs()
    
    # load data to annotate
    testdata = load_data(datadir)
    
    # encode dataset
    X = encode_words(testdata, idx)
    
    # tag sentences in dataset
    Y = model.predict(X)
    # get most likely tag for each word
    preds = []
    for s in Y:
        it = np.argmax(s)
        for key, item in idx['labels'].items():
            if item == it:
                preds.append(key)
    #Y = [[idx['labels'][np.argmax(y)] for y in s] for s in Y] # np.argmax(y) is not a key
    
    # extract entities and dump them to output file
    outf = open(outfile, "w")
    output_interactions(testdata, preds, outf)
    outf.close()
    
    # evaluate using official evaluator
    evaluate("DDI", datadir, outfile)

In [48]:
devel_path = '../../labAHLT/data/devel'
predict(devel_path, 'val.txt')

                   tp	  fp	  fn	#pred	#exp	P	R	F1
------------------------------------------------------------------------------
advise             85	 175	  53	 260	 138	32.7%	61.6%	42.7%
effect            174	 235	 141	 409	 315	42.5%	55.2%	48.1%
int                15	   0	  20	  15	  35	100.0%	42.9%	60.0%
mechanism         176	 207	  88	 383	 264	46.0%	66.7%	54.4%
------------------------------------------------------------------------------
M.avg            -	-	-	-	-	55.3%	56.6%	51.3%
------------------------------------------------------------------------------
m.avg             450	4219	 302	4669	 752	9.6%	59.8%	16.6%
m.avg(no class)   752	3917	   0	4669	 752	16.1%	100.0%	27.7%


In [49]:
test_path = '../../labAHLT/data/test'
predict(test_path, 'test.txt')

                   tp	  fp	  fn	#pred	#exp	P	R	F1
------------------------------------------------------------------------------
advise            118	 191	  94	 309	 212	38.2%	55.7%	45.3%
effect            170	 243	 113	 413	 283	41.2%	60.1%	48.9%
int                 7	   9	  11	  16	  18	43.8%	38.9%	41.2%
mechanism         205	 329	 132	 534	 337	38.4%	60.8%	47.1%
------------------------------------------------------------------------------
M.avg            -	-	-	-	-	40.4%	53.9%	45.6%
------------------------------------------------------------------------------
m.avg             500	5191	 350	5691	 850	8.8%	58.8%	15.3%
m.avg(no class)   850	4841	   0	5691	 850	14.9%	100.0%	26.0%


***